In [ ]:
numbers_to_remove = [79, 80, 81]
# adding columns for which there are NaNs in SARIMA
numbers_to_remove += [1, 4, 15, 16, 17, 18, 26, 28, 29, 40, 42, 47, 54, 59, 67, 71, 77]


In [ ]:
import pandas as pd
import glob
from utils import ends_with_number

# Define the folder where the CSV files are located
folder_path = r'D:\AGH\bankomaty_2022\data\redatyprognoz'

# Use glob to find all CSV files in the folder
file_pattern = folder_path + r'\prognozy_W_*.csv'
file_list = glob.glob(file_pattern)

# Create an empty list to hold the DataFrames
df_list = []

# Loop over each file and read it into a DataFrame
for file in file_list:
    df = pd.read_csv(file)
    period_id = df['date'].min()
    df['periodID'] = period_id
    df_list.append(df)

# Concatenate the DataFrames into a single DataFrame
xgb_raw_df = pd.concat(df_list)

# Get a list of column names to keep
columns_to_keep = [column for column in xgb_raw_df.columns if not ends_with_number(column, ['{:02d}'.format(num) for num in numbers_to_remove])]

# Remove the columns from the data frame
xgb_raw_df = xgb_raw_df[columns_to_keep]

In [ ]:
import os


# Define the directory where the Excel files are stored
directory = "D:/AGH/bankomaty_2022/data/Wielkość wypłat"

# Get a list of all Excel files in the directory
excel_files = [f for f in os.listdir(directory) if f.endswith(".xlsx")]

# Create an empty list to store the data frames
data_frames = []

# Loop through each Excel file and load the "prediction errors" sheet
for i, excel_file in enumerate(excel_files):
    file_path = os.path.join(directory, excel_file)
    xl = pd.ExcelFile(file_path)
    sheet_name = "prediction errors"
    df = xl.parse(sheet_name)
    period_id = os.path.splitext(excel_file)[0]
    df['periodID'] = period_id
    data_frames.append(df)

# Combine all data frames into a single data frame
sarima_raw_df = pd.concat(data_frames)

# Get a list of column names to keep
columns_to_keep = [column for column in sarima_raw_df.columns if not ends_with_number(column, ['{:02d}'.format(num) for num in numbers_to_remove])]

# Remove the columns from the data frame
sarima_raw_df = sarima_raw_df[columns_to_keep]

In [ ]:
import pandas as pd

# Define the file path of the Excel file
file_path = r'D:\AGH\bankomaty_2022\data\replaced 17-12-2022.xlsx'

# Read all sheets with names ending in "_errors" into a list of data frames
df_list = []
xl = pd.ExcelFile(file_path)
for sheet_name in xl.sheet_names:
    if sheet_name.endswith("_errors"):
        df = xl.parse(sheet_name)
        period_id = sheet_name[:-7]
        day, month, year = period_id.split("_")
        new_period_id = f"{year}_{month}_{day}"
        df['periodID'] = new_period_id
        df_list.append(df)

# Concatenate the data frames into a single data frame
bayes_raw_df = pd.concat(df_list)

# Rename the "Unnamed" column to "name"
bayes_raw_df = bayes_raw_df.rename(columns={"Unnamed: 0": "name"})

# Remove columns with names starting with 'ATM_N_'
bayes_raw_df = bayes_raw_df.loc[:, ~bayes_raw_df.columns.str.startswith('ATM_N_')]

# Print the resulting data frame
bayes_raw_df

# Get a list of column names to keep
columns_to_keep = [column for column in bayes_raw_df.columns if not ends_with_number(column, ['{:02d}'.format(num) for num in numbers_to_remove])]

# Remove the columns from the data frame
bayes_raw_df = bayes_raw_df[columns_to_keep]

In [ ]:
import pandas as pd

# Assuming bayes_raw_df is your original DataFrame

# Keep only rows with column name 'MAPE' or 'SMAPE'
bayes_filtered_df = bayes_raw_df[bayes_raw_df['name'].isin(['MAPE', 'SMAPE'])]

# Convert 'periodID' column from string to datetime format
bayes_filtered_df['periodID'] = pd.to_datetime(bayes_filtered_df['periodID'], format='%Y_%m_%d')

# Remove the row with '26.04.2019'
bayes_filtered_df = bayes_filtered_df[bayes_filtered_df['periodID'] != '2019-04-26']

bayes_filtered_df['model'] = 'BVAR'
# Reset index
bayes_filtered_df.reset_index(drop=True, inplace=True)

unique_period_ids = bayes_filtered_df['periodID'].unique()

# Sort the unique dates
sorted_unique_dates = sorted(unique_period_ids)

# Create a dictionary mapping the sorted unique dates to their rank
date_rank_mapping = {date: rank + 1 for rank, date in enumerate(sorted_unique_dates)}

# Apply the mapping to the 'periodID' column
bayes_filtered_df['periodID'] = bayes_filtered_df['periodID'].map(date_rank_mapping)


bayes_filtered_df

In [ ]:
import pandas as pd

# Assuming xgb_raw_df is your original DataFrame

# Keep only rows with column 'date' containing 'MAPE' or 'SMAPE'
xgb_filtered_df = xgb_raw_df[xgb_raw_df['date'].isin(['MAPE', 'SMAPE'])]

# Convert 'periodID' column from string to datetime format
xgb_filtered_df['periodID'] = pd.to_datetime(xgb_filtered_df['periodID'], format='%Y-%m-%d')

# Remove the rows with '26.04.2019' and '26.04.2020'
xgb_filtered_df = xgb_filtered_df[(xgb_filtered_df['periodID'] != '2019-04-26') & (xgb_filtered_df['periodID'] != '2020-04-26')]

# Reset index
xgb_filtered_df.reset_index(drop=True, inplace=True)

# Change column name 'date' to 'name'
xgb_filtered_df.rename(columns={'date': 'name'}, inplace=True)

# Get all column names starting with 'ATM_W_'
atm_columns = [col for col in xgb_filtered_df.columns if col.startswith('ATM_W_')]
xgb_filtered_df['model'] = 'XGB'

# Multiply all columns with the format 'ATM_W_XX' by 100
xgb_filtered_df[atm_columns] = xgb_filtered_df[atm_columns] * 100

unique_period_ids = xgb_filtered_df['periodID'].unique()

# Sort the unique dates
sorted_unique_dates = sorted(unique_period_ids)

# Create a dictionary mapping the sorted unique dates to their rank
date_rank_mapping = {date: rank + 1 for rank, date in enumerate(sorted_unique_dates)}

# Apply the mapping to the 'periodID' column
xgb_filtered_df['periodID'] = xgb_filtered_df['periodID'].map(date_rank_mapping)

xgb_filtered_df

In [ ]:
import pandas as pd
import re

# Assuming sarima_raw_df is your original DataFrame

# Keep only rows with column 'name' containing 'MAPE' or 'SMAPE'
sarima_filtered_df = sarima_raw_df[sarima_raw_df['name'].isin(['MAPE', 'SMAPE'])]

# Extract date from 'periodID' column
sarima_filtered_df['periodID'] = sarima_filtered_df['periodID'].apply(lambda x: x[6:16])

# Convert 'periodID' column from string to datetime format
sarima_filtered_df['periodID'] = pd.to_datetime(sarima_filtered_df['periodID'], format='%Y-%m-%d')

# Remove the rows with '26.04.2019' and '26.04.2020'
sarima_filtered_df = sarima_filtered_df[(sarima_filtered_df['periodID'] != '2018-08-01') & (sarima_filtered_df['periodID'] != '2019-08-01')]

sarima_filtered_df['model'] = 'SARIMA'
# Reset index
sarima_filtered_df.reset_index(drop=True, inplace=True)
unique_period_ids = sarima_filtered_df['periodID'].unique()

# Sort the unique dates
sorted_unique_dates = sorted(unique_period_ids)

# Create a dictionary mapping the sorted unique dates to their rank
date_rank_mapping = {date: rank + 1 for rank, date in enumerate(sorted_unique_dates)}

# Apply the mapping to the 'periodID' column
sarima_filtered_df['periodID'] = sarima_filtered_df['periodID'].map(date_rank_mapping)
sarima_filtered_df

In [ ]:
xgb_unique_dates = xgb_filtered_df['periodID'].unique()
bayes_unique_dates = bayes_filtered_df['periodID'].unique()
sarima_unique_dates = sarima_filtered_df['periodID'].unique()

xgb_unique_dates_sorted = sorted(xgb_unique_dates)
bayes_unique_dates_sorted = sorted(bayes_unique_dates)
sarima_unique_dates_sorted = sorted(sarima_unique_dates)

print("Sorted unique dates in XGB DataFrame:", xgb_unique_dates_sorted)
print("Sorted unique dates in Bayes DataFrame:", bayes_unique_dates_sorted)
print("Sorted unique dates in SARIMA DataFrame:", sarima_unique_dates_sorted)



In [ ]:
# Concatenate the three DataFrames
concatenated_df = pd.concat([xgb_filtered_df, bayes_filtered_df, sarima_filtered_df], axis=0)

# Reset the index of the concatenated DataFrame
concatenated_df.reset_index(drop=True, inplace=True)

# Get a list of columns to drop
columns_to_drop = [col for col in concatenated_df.columns if col.startswith('pred_W_')]

# Drop the columns
concatenated_df = concatenated_df.drop(columns_to_drop, axis=1)

concatenated_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Filter MAPE and SMAPE rows separately
mape_df = concatenated_df[concatenated_df['name'] == 'MAPE']
smape_df = concatenated_df[concatenated_df['name'] == 'SMAPE']

# Set the plot style
sns.set(style="whitegrid")

# Create the bar plots for MAPE and SMAPE
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10), sharey=True)

# MAPE plot
sns.barplot(x="model", y="value", hue="ATM", data=mape_df.melt(id_vars=["model", "name", "periodID"], var_name="ATM", value_name="value"), ax=ax1)
ax1.set_title("MAPE Comparison")
ax1.set_ylabel("MAPE")
ax1.set_xlabel("Model")
ax1.legend(title="ATM", loc="upper right", ncol=2)

# SMAPE plot
sns.barplot(x="model", y="value", hue="ATM", data=smape_df.melt(id_vars=["model", "name", "periodID"], var_name="ATM", value_name="value"), ax=ax2)
ax2.set_title("SMAPE Comparison")
ax2.set_ylabel("SMAPE")
ax2.set_xlabel("Model")
ax2.legend(title="ATM", loc="upper right", ncol=2)

# Show the plot
plt.show()



In [ ]:
import numpy as np

# Calculate the mean MAPE and SMAPE for each model and time period
mape_mean = mape_df.groupby(['model', 'periodID']).mean()
smape_mean = smape_df.groupby(['model', 'periodID']).mean()

# Create bar plots for mean MAPE and SMAPE
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

# MAPE plot
sns.barplot(x="model", y="value", hue="periodID", data=mape_mean.reset_index().melt(id_vars=["model", "periodID"], value_name="value"), ax=ax1)
ax1.set_title("Mean MAPE Comparison")
ax1.set_ylabel("Mean MAPE")
ax1.set_xlabel("Model")
ax1.legend(title="Time Period", loc="upper right")

# SMAPE plot
sns.barplot(x="model", y="value", hue="periodID", data=smape_mean.reset_index().melt(id_vars=["model", "periodID"], value_name="value"), ax=ax2)
ax2.set_title("Mean SMAPE Comparison")
ax2.set_ylabel("Mean SMAPE")
ax2.set_xlabel("Model")
ax2.legend(title="Time Period", loc="upper right")

# Save the plot
plt.savefig(r'D:\AGH\bankomaty_2022\pics\mape_smape_comparison.png', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Select a subset of ATMs
atm_subset = ['ATM_W_02', 'ATM_W_03', 'ATM_W_05']

# Filter the dataframe
mape_subset = mape_df[atm_subset + ['model', 'periodID']]
smape_subset = smape_df[atm_subset + ['model', 'periodID']]

# Create bar plots for mean MAPE and SMAPE
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

# MAPE plot
sns.barplot(x="model", y="value", hue="periodID", data=mape_subset.reset_index().melt(id_vars=["model", "periodID"], value_name="value"), ax=ax1)
ax1.set_title("Mean MAPE Comparison")
ax1.set_ylabel("Mean MAPE")
ax1.set_xlabel("Model")
ax1.legend(title="Time Period", loc="upper left")

# SMAPE plot
sns.barplot(x="model", y="value", hue="periodID", data=smape_subset.reset_index().melt(id_vars=["model", "periodID"], value_name="value"), ax=ax2)
ax2.set_title("Mean SMAPE Comparison")
ax2.set_ylabel("Mean SMAPE")
ax2.set_xlabel("Model")
ax2.legend(title="Time Period", loc="upper left")


# Show the plot
plt.show()


In [ ]:
has_nans = concatenated_df.isna().any().any()
print(has_nans)

In [ ]:
# Create a list of unique time periods
time_periods = concatenated_df["periodID"].unique()

# Loop through the time periods and create a bar chart for each one
for period in time_periods:
    # Filter the data for the current time period
    mape_period = mape_df[mape_df["periodID"] == period]
    smape_period = smape_df[smape_df["periodID"] == period]

    # Create bar plots for the current time period
    # (Use the same plotting code as before, but replace 'mape_df' with 'mape_period' and 'smape_df' with 'smape_period')
    # Filter the dataframe
    mape_subset = smape_period[atm_subset + ['model', 'periodID']]
    smape_subset = smape_period[atm_subset + ['model', 'periodID']]

    # Create bar plots for mean MAPE and SMAPE
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

    # MAPE plot
    sns.barplot(x="model", y="value", hue="periodID", data=mape_subset.reset_index().melt(id_vars=["model", "periodID"], value_name="value"), ax=ax1)
    ax1.set_title("Mean MAPE Comparison")
    ax1.set_ylabel("Mean MAPE")
    ax1.set_xlabel("Model")
    ax1.legend(title="Time Period", loc="upper right")

    # SMAPE plot
    sns.barplot(x="model", y="value", hue="periodID", data=smape_subset.reset_index().melt(id_vars=["model", "periodID"], value_name="value"), ax=ax2)
    ax2.set_title("Mean SMAPE Comparison")
    ax2.set_ylabel("Mean SMAPE")
    ax2.set_xlabel("Model")
    ax2.legend(title="Time Period", loc="upper right")

    # Show the plot
    plt.show()

In [ ]:
# Calculate mean MAPE and SMAPE for each model and period across all ATMs
mean_mape = mape_df.groupby(['model', 'periodID']).mean().stack().reset_index(name='value')
mean_mape['metric'] = 'MAPE'

mean_smape = smape_df.groupby(['model', 'periodID']).mean().stack().reset_index(name='value')
mean_smape['metric'] = 'SMAPE'

# Combine the mean MAPE and SMAPE DataFrames
mean_metrics_df = pd.concat([mean_mape, mean_smape], ignore_index=True)

# Pivot the combined dataframe for all ATMs
all_atms_comparison_table = mean_metrics_df.pivot_table(index=['metric', 'model'], columns='periodID', values='value').round(2)

# Save the table to a CSV file
all_atms_comparison_table.to_csv(r'D:\AGH\bankomaty_2022\results\all_atms_model_comparison.csv')



In [ ]:
import pandas as pd

# Filter only rows with 'MAPE' metric
mape_df = concatenated_df[concatenated_df['name'] == 'MAPE']

# Calculate the rank for each model based on MAPE for each ATM and period
atm_columns = mape_df.columns[1:-2]
mape_df_rank = mape_df.set_index(['model', 'periodID'])[atm_columns].stack().reset_index(name='MAPE').rename(columns={'level_2': 'ATM'})
mape_df_rank['rank'] = mape_df_rank.groupby(['ATM', 'periodID'])['MAPE'].rank(ascending=True, method='dense')

# Save the ranking table to a CSV file
mape_df_rank.to_csv('D:/AGH/bankomaty_2022/results/rank_table_ATM_period.csv', index=False)

# Count the number of times each model had rank 1, 2, and 3
rank_count = mape_df_rank.groupby(['model', 'rank']).size().reset_index(name='count')

# Pivot the rank count table to have model as index and rank as columns
rank_count_pivot = rank_count.pivot_table(index='model', columns='rank', values='count').fillna(0)

# Save the summary table to a CSV file
rank_count_pivot.to_csv('D:/AGH/bankomaty_2022/results/summary_rank_count_ATM_period.csv')


In [ ]:
import pandas as pd

# Filter only rows with 'SMAPE' metric
smape_df = concatenated_df[concatenated_df['name'] == 'SMAPE']

# Calculate the rank for each model based on SMAPE for each ATM and period
atm_columns = smape_df.columns[1:-2]
smape_df_rank = smape_df.set_index(['model', 'periodID'])[atm_columns].stack().reset_index(name='SMAPE').rename(columns={'level_2': 'ATM'})
smape_df_rank['rank'] = smape_df_rank.groupby(['ATM', 'periodID'])['SMAPE'].rank(ascending=True, method='dense')

# Save the ranking table to a CSV file
smape_df_rank.to_csv('D:/AGH/bankomaty_2022/results/rank_table_ATM_period_SMAPE.csv', index=False)

# Count the number of times each model had rank 1, 2, and 3
smape_rank_count = smape_df_rank.groupby(['model', 'rank']).size().reset_index(name='count')

# Pivot the rank count table to have model as index and rank as columns
smape_rank_count_pivot = smape_rank_count.pivot_table(index='model', columns='rank', values='count').fillna(0)

# Save the summary table to a CSV file
smape_rank_count_pivot.to_csv('D:/AGH/bankomaty_2022/results/summary_rank_count_ATM_period_SMAPE.csv')


In [ ]:
smape_rank_count_pivot

In [ ]:
import pandas as pd

period_pairs = [(1, 5), (2, 6), (3, 7), (4, 8)]
models = ['BVAR', 'SARIMA', 'XGB']
concatenated_mape = concatenated_df[concatenated_df['name'] == "MAPE"]
atms = concatenated_mape.columns[1:-2]

transition_matrices = {}

for period_pair in period_pairs:
    early_period, later_period = period_pair
    
    early_period_df = concatenated_mape[concatenated_mape['periodID'] == early_period]
    later_period_df = concatenated_mape[concatenated_mape['periodID'] == later_period]
    
    transition_matrix = pd.DataFrame(index=models, columns=models, data=0)
    
    for atm in atms:
        early_best_model = early_period_df.loc[early_period_df[atm].idxmin()]['model']
        later_best_model = later_period_df.loc[later_period_df[atm].idxmin()]['model']
        
        transition_matrix.at[early_best_model, later_best_model] += 1
    
    transition_matrices[period_pair] = transition_matrix

for period_pair, transition_matrix in transition_matrices.items():
    print(f"Transition matrix for periods {period_pair} (MAPE):\n{transition_matrix}\n")

In [ ]:
import pandas as pd
import openpyxl

# The code for generating the transition_matrices should be placed here.

# Create a new Excel workbook
writer = pd.ExcelWriter(r'D:\AGH\bankomaty_2022\results\matrices_mape.xlsx', engine='openpyxl')

# Write each transition matrix to a separate sheet in the workbook
for period_pair, transition_matrix in transition_matrices.items():
    sheet_name = f"Periods {period_pair[0]}-{period_pair[1]}"
    transition_matrix.to_excel(writer, sheet_name=sheet_name)

# Save the Excel workbook
writer.save()



In [ ]:
import pandas as pd

period_pairs = [(1, 5), (2, 6), (3, 7), (4, 8)]
models = ['BVAR', 'SARIMA', 'XGB']
concatenated_smape = concatenated_df[concatenated_df['name'] == "SMAPE"]
atms = concatenated_smape.columns[1:-2]

transition_matrices = {}

for period_pair in period_pairs:
    early_period, later_period = period_pair
    
    early_period_df = concatenated_smape[concatenated_smape['periodID'] == early_period]
    later_period_df = concatenated_smape[concatenated_smape['periodID'] == later_period]
    
    transition_matrix = pd.DataFrame(index=models, columns=models, data=0)
    
    for atm in atms:
        early_best_model = early_period_df.loc[early_period_df[atm].idxmin()]['model']
        later_best_model = later_period_df.loc[later_period_df[atm].idxmin()]['model']
        
        transition_matrix.at[early_best_model, later_best_model] += 1
    
    transition_matrices[period_pair] = transition_matrix

for period_pair, transition_matrix in transition_matrices.items():
    print(f"Transition matrix for periods {period_pair} (MAPE):\n{transition_matrix}\n")

In [ ]:
import pandas as pd
import openpyxl

# The code for generating the transition_matrices should be placed here.

# Create a new Excel workbook
writer = pd.ExcelWriter(r'D:\AGH\bankomaty_2022\results\matrices_smape.xlsx', engine='openpyxl')

# Write each transition matrix to a separate sheet in the workbook
for period_pair, transition_matrix in transition_matrices.items():
    sheet_name = f"Periods {period_pair[0]}-{period_pair[1]}"
    transition_matrix.to_excel(writer, sheet_name=sheet_name)

# Save the Excel workbook
writer.save()
